In [1]:
import gql
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
import pandas as pd
import asyncio

# Define transport variable
transport = AIOHTTPTransport(url="https://sui-testnet.mystenlabs.com/graphql")

#Mainnet: https://sui-mainnet.mystenlabs.com/graphql
#Testnet: https://sui-testnet.mystenlabs.com/graphql

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

In [102]:
# Define your parameters
objectID = "0x8f4ba9c068cc90c1140d7310ca81b8db1f43ec38cfb252ec227ac307d8829c8d"
last = 10


def get_query_string(objectID, last, before=None):
    before_clause = f', before: "{before}"' if before else ''
    return f"""
    query get_digests {{
      transactionBlocks(filter: {{inputObject: "{objectID}"}}, last: {last}{before_clause}) {{
        edges {{
          node {{
            digest
            sender {{
              address
            }}
          }}
        }}
        pageInfo {{
          hasNextPage
          endCursor
          hasPreviousPage
          startCursor
        }}
      }}
    }}"""

In [ ]:
async def fetch_data(query_string):
    query = gql(query_string)
    result = await client.execute_async(query)
    return result['transactionBlocks']

# Initial query execution
before = None
query_string = get_query_string(objectID, last, before)
response_data = await fetch_data(query_string)

# Initialize the count
call_count = 1

# Define the df to store digests
df = pd.DataFrame(columns=["digest", "sender_address"])

# Extract and print the relevant data
for edge in response_data['edges']:
    digest = edge['node']['digest']
    sender_address = edge['node']['sender']['address']
    new_row = pd.DataFrame([{"digest": digest, "sender_address": sender_address}])
    df = pd.concat([df, new_row], ignore_index=True)
    # print(f"Digest: {digest}, Sender Address: {sender_address}")

# Pagination logic: iterate while hasPreviousPage is True
page_info = response_data['pageInfo']
while page_info['hasPreviousPage'] and call_count < 100:
    before = page_info['endCursor']
    query_string = get_query_string(objectID, last, before)
    response_data = await fetch_data(query_string)

    # Increment the count
    call_count += 1

    # Extract and print the relevant data for the next page
    for edge in response_data['edges']:
        digest = edge['node']['digest']
        sender_address = edge['node']['sender']['address']
        new_row = pd.DataFrame([{"digest": digest, "sender_address": sender_address}])
        df = pd.concat([df, new_row], ignore_index=True)
        # print(f"Digest: {digest}, Sender Address: {sender_address}")

    page_info = response_data['pageInfo']

    print(call_count)

    print(response_data)

    # Wait 5 seconds every 50 iterations
    if call_count % 50 == 0:
        await asyncio.sleep(5)

print(f"No more pages. Total number of calls: {call_count}")

In [ ]:
display(df)
# Save the DataFrame as a JSON file in the data folder
df.to_json('data/digests.json', orient='records', lines=True)

,digest,sender_address
0,EyKYy7fv3XyDMR2R4wHpGbDWfBtboeBWmHxybX1TxsE4,0x15415574adb9bf4fe9e6d93291d6d7256d06ccc8bcf7...
1,EQMPKtdqee4dAKvyaSH7JpU95o622peYinhydNw72Lbe,0x65391674eb4210940ea98ae451237d9335920297e7c8...
2,3CwH6TVFVAm7AhFo7fSaoooy7TQpRE638ZCNvx8cufiW,0x15415574adb9bf4fe9e6d93291d6d7256d06ccc8bcf7...
3,BjP6oFDfXqHuYc1mv8FFBm5Wp9Xt58FqX66JNc1dyXuD,0x65391674eb4210940ea98ae451237d9335920297e7c8...
4,DHi6KTA5pz1Hn6R8pTKgwvavaFnSdR6zszocgUxgJJM6,0x15415574adb9bf4fe9e6d93291d6d7256d06ccc8bcf7...
...,...,...
995,AxCHRU9VG5BtUF7oyaxVLCx5umGjjd6bHZpGCUPaH713,0x65391674eb4210940ea98ae451237d9335920297e7c8...
996,85aLyYXayszxcD8ocZrnRyVVuEH6ZpFmReXWQ6ou3pk3,0xeca59f830c0929f751eb2325d5ac7a673c0899a1f5e9...
997,ESTdAWVYuaUGqyw1zPK2wXSNgCjsRXnMCLpEMxsmz1cL,0x65391674eb4210940ea98ae451237d9335920297e7c8...
998,vi3125pxJXGT9uGr1cXQTuNit6csLnAiHEwoaaow342,0xeca59f830c0929f751eb2325d5ac7a673c0899a1f5e9...
